### Import Libraries

In [1]:
import os
import glob

import cv2
import numpy as np

import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, random_split
# from torchvision.models import resnet50, vit_b_16
# from torchvision import transforms
# from torchmetrics import Accuracy
from torchinfo import summary

from utilities import AITEX
from model_architectures import VAE, MiniUNet


### Get Data

In [2]:
defect_codes = {
    0: "Normal",
    2:	"Broken end",
    6:	"Broken yarn",
    10:	"Broken pick",
    16:	"Weft curling",
    19:	"Fuzzyball",
    22:	"Cut selvage",
    23:	"Crease",
    25:	"Warp ball",
    27:	"Knots",
    29:	"Contamination",
    30: "Nep",
    36:	"Weft crack",
}

class AITEXPatched(AITEX):
    def __init__(self, *args, normal_only=False, defect_only=False, **kwargs,):
        super(AITEXPatched, self).__init__(*args, **kwargs)

        self.patched_images = []
        self.patched_masks = []
        self.has_defect = []
        for index, img in enumerate(self.images):
            img_new = cv2.resize(img, (4096, 256)) / 255. 
            self.patched_images.extend([img_new[:,i:i+256] for i in range(0, 4096, 256)])

            mask_new = cv2.resize(self.masks[index], (4096, 256))
            mask_patches = [mask_new[:,i:i+256] for i in range(0, 4096, 256)]
            self.patched_masks.extend(mask_patches)

            self.has_defect.extend([1 if np.sum(x) > 0 else 0 for x in mask_patches])
        
        if normal_only:
            indices = [x for x, y in enumerate(self.has_defect) if y==0]
            self.patched_images = [self.patched_images[x] for x in indices]
            self.patched_masks = [self.patched_masks[x] for x in indices]
            self.has_defect = [self.has_defect[x] for x in indices]
        if defect_only:
            indices = [x for x, y in enumerate(self.has_defect) if y!=0]
            self.patched_images = [self.patched_images[x] for x in indices]
            self.patched_masks = [self.patched_masks[x] for x in indices]
            self.has_defect = [self.has_defect[x] for x in indices]
    
    def __len__(self):
        """Get length of full dataset."""
        return len(self.patched_images)    
    
    def __getitem__(self, idx):
        """Return specific index of dataset."""
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img = self.patched_images[idx]
        
        return np.stack([img, img, img], axis=0), self.has_defect[idx]

In [8]:
root = os.path.abspath(os.path.join(os.getcwd(), ".."))
data_dir = os.path.join(root, "data")
aitex_dir = os.path.join(data_dir, "aitex")

data = AITEXPatched(aitex_dir, greyscale=True)#, normal_only=True)
num_samples = len(data)
train_samples = int(num_samples * 0.9)
val_samples = num_samples - train_samples
train, val = random_split(data, [train_samples, val_samples])

# bs = 32
# train_loader = DataLoader(train, batch_size=bs, shuffle=True)
# val_loader = DataLoader(val, batch_size=bs, shuffle=True)

### Experiment with ViT

In [4]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from datasets import load_metric

metric = load_metric("accuracy")
labels = {0: "normal", 1: "defect"}

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224', 
    num_labels=2,
    id2label=labels, 
    label2id={y: x for x, y in labels.items()},
    ignore_mismatched_sizes=True
)

c:\compute\envs\fdd\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Jad Raad\AppData\Local\Temp\ipykernel_31380\1585744307.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
c:\compute\envs\fdd\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.weig

In [32]:
def process_example(example):
    inputs = feature_extractor(example[0], return_tensors="pt")
    inputs['pixel_values'] = inputs['pixel_values'].reshape((3, 224, 224))
    inputs["label"] = example[1]
    return inputs

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

train_ds = [process_example(x) for x in train]
val_ds = [process_example(x) for x in val]

In [40]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-base-beans",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [41]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=feature_extractor,
)

In [42]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)

  0%|          | 0/444 [07:56<?, ?it/s]
c:\compute\envs\fdd\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
c:\compute\envs\fdd\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
c:\compute\envs\fdd\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
                                                 
 23%|██▎       | 100/444 [03:17<01:40,  3.41it/s]

{'loss': 0.2558, 'learning_rate': 0.0001954954954954955, 'epoch': 0.09}


                                                 
 23%|██▎       | 100/444 [03:20<01:40,  3.41it/s]

{'loss': 0.2379, 'learning_rate': 0.000190990990990991, 'epoch': 0.18}


                                                 
 23%|██▎       | 100/444 [03:23<01:40,  3.41it/s]

...

{'loss': 0.2704, 'learning_rate': 6.306306306306306e-06, 'epoch': 3.87}


                                                 
 23%|██▎       | 100/444 [05:42<01:40,  3.41it/s]

{'loss': 0.119, 'learning_rate': 1.801801801801802e-06, 'epoch': 3.96}


                                                 
100%|██████████| 444/444 [02:29<00:00,  2.98it/s]


{'train_runtime': 149.195, 'train_samples_per_second': 94.963, 'train_steps_per_second': 2.976, 'train_loss': 0.2015069006262599, 'epoch': 4.0}
***** train metrics *****
  epoch                    =        4.0
  train_loss               =     0.2015
  train_runtime            = 0:02:29.19
  train_samples_per_second =     94.963
  train_steps_per_second   =      2.976


In [44]:
train_results.metrics

{'train_runtime': 149.195,
 'train_samples_per_second': 94.963,
 'train_steps_per_second': 2.976,
 'train_loss': 0.2015069006262599,
 'epoch': 4.0}